In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from pathlib import Path
from skimage import io
import numpy as np
import math
import random
from yugioh.card_dataset import CardDataset

import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

from shapely.geometry import Polygon, Point
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
card_dataset = CardDataset("input/yugioh-cards/")

In [ ]:
card_dataset[0]

In [ ]:
plt.imshow(card_dataset[0].image())

In [ ]:
dtd_dir="input/dtd-r1.0.1/"
background_paths = list(Path(dtd_dir).glob("**/*.jpg"))
bg_images=[]
for image_path in random.sample(background_paths, 100):
    bg_images.append(io.imread(image_path))

In [ ]:
image_transformations = iaa.Sequential([
    iaa.Multiply((0.7, 1.5)), 
    iaa.AddElementwise((-20, 20), per_channel=0.5),
])

spatial_transformations = iaa.Sequential([
    iaa.Multiply((0.5, 1.5)), # change brightness, doesn't affect keypoints
    iaa.Affine(
        scale=0.6,
        translate_percent={"x": (-0.4, 0.4), "y": (-0.4, 0.4)},
        rotate=(-90, 90),
    ),
])

class Scenes:
    
    pad_factor = 100
    card_shape = (614, 421, 3)
    card_key_points = ia.KeypointsOnImage([
        ia.Keypoint(x=0, y= 0),
        ia.Keypoint(x=0, y=card_shape[0]),   
        ia.Keypoint(x=card_shape[1], y=card_shape[0]),
        ia.Keypoint(x=card_shape[1], y= 0)
    ], shape=card_shape)
    
    def __init__(self, backgrounds, scene_width=700, scene_height=700):
        self.backgrounds = backgrounds
        self.scene_width = scene_width
        self.scene_height = scene_height
        
        aux_w = scene_width - Scenes.card_shape[1]
        self.card_w_padding = [aux_w // 2] * 2
        self.card_w_padding[1] += aux_w % 2
        
        aux_w = scene_height - Scenes.card_shape[0]
        self.card_h_padding = [aux_w // 2] * 2
        self.card_h_padding[1] += aux_w % 2
        
    def _generate_random_background(self):
        selected_bg = random.choice(self.backgrounds)
        bg_height, bg_width, _ = selected_bg.shape
        repeat_w = math.ceil(self.scene_width/bg_width)
        repeat_h = math.ceil(self.scene_height/bg_height)
        background = np.repeat(selected_bg, repeats=repeat_w, axis=1)
        background = np.repeat(background, repeats=repeat_h, axis=0)
        return background[:self.scene_height,:self.scene_width,:]
    
    @staticmethod
    def kps_to_polygon(kps):
        pts=[(kp.x,kp.y) for kp in kps]
        return Polygon(pts)
    
    def kps_to_bounding(self, kps):
        """
            Determine imgaug bounding box from imgaug keypoints
        """
        extend=3 # To make the bounding box a little bit bigger
        kpsx=[kp.x for kp in kps.keypoints]
        minx=max(0,int(min(kpsx)-extend))
        maxx=min(self.scene_width,int(max(kpsx)+extend))
        kpsy=[kp.y for kp in kps.keypoints]
        miny=max(0,int(min(kpsy)-extend))
        maxy=min(self.scene_height,int(max(kpsy)+extend))
        if minx==maxx or miny==maxy:
            return None
        else:
            return ia.BoundingBox(x1=minx,y1=miny,x2=maxx,y2=maxy)
    
    def augment_card(self, card, draw_kps=False):
        transformed_img = image_transformations(image=card)
        transformed_img = np.pad(transformed_img, [
            self.card_h_padding,
            self.card_w_padding, 
            (0,0)
        ])
        kps = Scenes.card_key_points.shift(self.card_w_padding[0], self.card_h_padding[0])
        kps.shape = transformed_img.shape
        transformed_img, transformed_kps = spatial_transformations(image=transformed_img, keypoints=kps)
        if draw_kps:
            transformed_img = transformed_kps.draw_on_image(transformed_img, size=20)
        return transformed_img[:self.scene_height,:self.scene_width,:], transformed_kps
        
    def generate(self, cards, draw_kps=False):
        full_image = self._generate_random_background()
        keypoint_list = []
        original_polys = []
        modified_polys = []
        bounding_boxes = []
        for card in cards:
            card_image, keypoints = self.augment_card(card.image(), draw_kps=draw_kps)
            current_poly = Scenes.kps_to_polygon(keypoints)
            for idx, old_poly in enumerate(modified_polys):
                if old_poly.overlaps(current_poly):
                    new_poly = old_poly - current_poly
                    if isinstance(new_poly, MultiPolygon):
                        new_poly = sorted(new_poly, key=lambda pol: pol.area)[1]
                    modified_polys[idx] = new_poly
            keypoint_list.append(keypoints)
            original_polys.append(current_poly)
            modified_polys.append(current_poly)
            bounding_boxes.append(self.kps_to_bounding(keypoints))
            
            full_image = np.where(card_image, card_image, full_image)
            
        return (
            full_image,
            keypoints,
            original_polys,
            modified_polys,
            bounding_boxes,
        )
            

scenes = Scenes(bg_images, scene_width=1920, scene_height=1080)
card_ids = random.sample(range(len(card_dataset)), 5)

cards = random.sample(card_dataset,3)
board, keypoints, polys, mod, bounding_boxes = scenes.generate(cards, draw_kps=True)


In [ ]:
fig = plt.figure(dpi=200)
ax = fig.gca()
ax.imshow(board)
rects = []
for idx, (card, bb, poly) in enumerate(zip(cards,bounding_boxes, mod)):
    rect = Rectangle((bb.x1, bb.y1),bb.x2-bb.x1, bb.y2-bb.y1,facecolor='red', fill=False, alpha=1)
    ax.text(bb.x1, bb.y1, f"{idx} - {card.name}", size=7)
    ax.plot(*poly.exterior.xy)
    rects.append(rect)

# Create patch collection with specified colour/alpha
pc = PatchCollection(rects, match_original=True)

# Add collection to axes
ax.add_collection(pc)